__❗This notebook is based on the Ethan Rosenthal's blogpost [here](https://www.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/).__

## Imports

In [1]:
import numpy as np
import pandas as pd
import os

## Reading the data

In [2]:
DATA_PATH = "../data/ml-100k/"

names = ["user_id", "item_id", "rating", "timestamp"]

df = pd.read_csv(os.path.join(DATA_PATH, "u.data"), names=names, sep='\t')
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]

print(f"There are {n_users} users and {n_items} items")

There are 943 users and 1682 items


Let's make numpy matrix with ratings

In [4]:
ratings = np.zeros((n_users, n_items))
for _, row in df.iterrows():
    ratings[row.user_id - 1, row.item_id - 1] = row.rating

ratings

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

## Train-test split

Let's calculate sparcity of the ratings matrix.

In [5]:
sparcity = np.sum(ratings != 0) / np.prod(ratings.shape) * 100
print("Sparcity: {:4.2f}%".format(sparcity))

Sparcity: 6.30%


This means that only 6.3% of data has non-zero elements.

## Train-test split

In [6]:
def train_test_split(ratings: np.ndarray):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], size=10, replace=False)
        train[user, test_ratings] = 0
        test[user, test_ratings] = ratings[user, test_ratings]
        
    assert np.all((train * test) == 0)
    return train, test

In [7]:
train, test = train_test_split(ratings)

## Collaborative filtering

There are 2 types of collaborative filtering: user- and item-based CF. In both cases the similarity function $sim(x, y)$ is calculated which is used for selecting the most similars users/items.

### Similarity measures

#### Cosine similarity

$$
sim(u,v)=\dfrac{\langle r_u,\, r_v \rangle}{\lVert r_u \rVert \lVert r_v \rVert},
$$  
where $r_u,\, r_v$ are ratings of $u$ and $v$ users respectively. The same method is also working for item-item.

In [8]:
def cosine_similarity(ratings: np.ndarray, kind="item", eps=1e-6):
    if kind == "item":
        norms = np.linalg.norm(ratings, axis=0) + eps
        return np.matmul(ratings.T, ratings) / norms / norms[None, :]
    elif kind == "user":
        norms = np.linalg.norm(ratings, axis=1) + eps
        return np.matmul(ratings, ratings.T) / norms / norms[None, :]

#### Jaccard index

This metric is often used in segmentation tasks

$$
sim(u,\, v)= \dfrac{|I_u \cap I_v|}{|I_u \cup I_v|},
$$  
where $I_u$ and $I_v$ are sets of rated items for users $u$ and $v$ respectively.

In [18]:
def jaccard_similarity(ratings: np.ndarray, kind="item", eps=1e-6):
    
    if kind == "item":
        similarity = np.zeros((ratings.shape[1], ratings.shape[1]))
        
        for i in range(len(similarity)):
            for j in range(i, len(similarity)):
                denom = (ratings[:, i] + ratings[:, j] > 0).sum() + eps
                similarity[i][j] = (ratings[:, i] * ratings[:, j] > 0).sum() / denom
                similarity[j][i] = similarity[i][j]
    elif kind == "user":
        similarity = np.zeros((ratings.shape[0], ratings.shape[0]))
        
        for i in range(len(similarity)):
            for j in range(i, len(similarity)):
                denom = (ratings[i] + ratings[j] > 0).sum() + eps
                similarity[i][j] = (ratings[i] * ratings[j] > 0).sum() / denom
                similarity[j][i] = similarity[i][j]
    
    return similarity

#### Dot product similarity

$$
sim(u,\, v) = \langle u,\, v \rangle
$$

In [20]:
def dot_product_similarity(ratings: np.ndarray, kind="item"):
    if kind == "item":
        return ratings.T.dot(ratings)
    elif kind == "user":
        return ratings.dot(ratings.T)

#### Pearson correlation

In [25]:
def pearson_similarity(ratings: np.ndarray, kind="item", eps=1e-6):
    mask = ratings > 0
    if kind == "user":
        mean_ratings = ratings.mean(axis=1)
        
        ratings_centralized = (ratings - mean_ratings[:, None]) * mask
        ratings_centralized = ratings_centralized.dot(ratings_centralized.T)
    elif kind == "item":
        mean_ratings = ratings.mean(axis=0)
        
        ratings_centralized = (ratings - mean_ratings[None, :]) * mask
        ratings_centralized = ratings_centralized.T.dot(ratings_centralized)
        
    denom = np.linalg.norm(ratings_centralized, axis=1) + eps
    return ratings_centralized / denom[:, None] / denom[None, :]

### Training

Now, let's predict item $i$ rating for user $u$:
$$
\hat{r}_{ui} = \dfrac{\sum_{v\in U}sim(u,\, v)\cdot r_{vi}}{\sum_{v\in U}|sim(u,\, v)|},
$$

In [60]:
ratings.sum(axis=0).shape

(1682,)

#### Prediction

In [68]:
def predict(ratings: np.ndarray, similarity: np.ndarray, kind="item", eps=1e-5):
    denom = np.abs(similarity).sum(axis=1) + eps
    if kind == "user":
        assert similarity.shape[0] == ratings.shape[0], f"{similarity.shape}, {ratings.shape}"
        
        return similarity.dot(ratings) / denom[:, None]
    elif kind == "item":
        assert similarity.shape[0] == ratings.shape[1], f"{similarity.shape}, {ratings.shape}"
        
        return ratings.dot(similarity) / denom[None, :]

In [73]:
item_prediction = predict(train, item_similarity, kind="item")
user_prediction = predict(train, user_similarity, kind="user")

item_prediction[:4, :4]

array([[0.17488152, 0.6178573 , 0.94690924, 0.38254766],
       [0.0379716 , 0.0929734 , 0.1885205 , 0.06397883],
       [0.01276649, 0.03836561, 0.06985152, 0.02393658],
       [0.01014075, 0.03099916, 0.05517531, 0.01924283]])

#### Testing

Let's define a loss function

In [77]:
def mse(prediction, true):
    prediction = prediction[true.nonzero()].flatten()
    true = true[true.nonzero()].flatten()
    return np.mean((prediction - true) ** 2)

In [82]:
print(f"User-based CF MSE: {mse(user_prediction, test)}")
print(f"Item-based CF MSE: {mse(item_prediction, test)}")

User-based CF MSE: 9.763106529769319
Item-based CF MSE: 16.98220326913952


### Bias-substracted Collaborative Filtering

As users can rate movies in different ranges, we want to track only deviation of current user's rating and his normal rating.

$$
\hat{r}_{ui}=\overline{r}_u + \dfrac{\sum_{v\in U}sim(u,\, v)(r_{vi} - \overline{r}_v)}{\sum_{v\in U}|sim(u,\, v)|}
$$

#### Prediction

In [83]:
ratings.mean(axis=0).shape

(1682,)

In [87]:
def bias_substracted_predict(ratings: np.ndarray, similarity: np.ndarray, kind="item", eps=1e-5):
    denom = np.abs(similarity).sum(axis=1) + eps
    
    if kind == "item":
        assert similarity.shape[0] == ratings.shape[1], f"{similarity.shape}, {ratings.shape}"
        
        mean_ratings = ratings.mean(axis=0)
        
        return (ratings - mean_ratings[None, :]).dot(similarity) / denom[None, :] + mean_ratings[None, :]
    elif kind == "user":
        assert similarity.shape[0] == ratings.shape[0], f"{similarity.shape}, {ratings.shape}"
        mean_ratings = ratings.mean(axis=1)
        
        return similarity.dot(ratings - mean_ratings[:, None]) / denom[:, None] + mean_ratings[:, None]

In [88]:
item_prediction = bias_substracted_predict(train, item_similarity, kind="item")
user_prediction = bias_substracted_predict(train, user_similarity, kind="user")

item_prediction[:4, :4]

array([[1.74667644, 0.82382754, 0.9039091 , 0.99671295],
       [1.60976652, 0.29894365, 0.14552036, 0.67814412],
       [1.58456141, 0.24433586, 0.02685137, 0.63810187],
       [1.58193567, 0.2369694 , 0.01217516, 0.63340812]])

#### Testing

In [90]:
print(f"Bias-substracted User-based CF MSE: {mse(user_prediction, test)}")
print(f"Bias-substracted Item-based CF MSE: {mse(item_prediction, test)}")

Bias-substracted User-based CF MSE: 9.56326833738522
Bias-substracted Item-based CF MSE: 11.517464593700096
